# Avaliação de Modelos de IA para Consultas Jurídicas Brasileiras

Este notebook permite executar o pipeline de avaliação de modelos de IA especializados em direito brasileiro. Ele inclui setup completo, configuração de cache e execução intuitiva baseada no script `run.py`.

## Pré-requisitos
- Conta no OpenRouter para API key
- Ambiente Colab ou Jupyter com acesso a GPU (recomendado)

## Passos do Setup
1. Definir API key do OpenRouter
2. Clonar repositório
3. Instalar dependências
4. Configurar cache do Hugging Face
5. Executar avaliação

## 1. Definir API Key do OpenRouter

Execute a célula abaixo e insira sua API key quando solicitado. Ela será armazenada como variável de ambiente.

In [ ]:
import os

# Solicitar API key do usuário
api_key = input("Digite sua API key do OpenRouter: ")
os.environ['OPENAI_API_KEY'] = api_key
print("API key definida com sucesso!")

## 2. Clonar Repositório

Clone o repositório oficial do projeto.

In [ ]:
!git clone https://github.com/expagepay/EvalAItoNomartiveConsults.git
%cd /content/EvalAItoNomartiveConsults

## 3. Instalar Dependências

Instale todas as bibliotecas necessárias do arquivo `requirements.txt`.

In [ ]:
!pip install -r requirements.txt

## 4. Configurar Cache do Hugging Face

Configure o cache para otimizar downloads de modelos e evitar re-downloads.

In [ ]:
import os

# Configurar caminhos de cache (adaptado para Colab/Linux)
cache_dir = '/content/HF_Cache'
os.makedirs(cache_dir, exist_ok=True)

os.environ['HF_HUB_CACHE'] = cache_dir
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'
os.environ['TRANSFORMERS_CACHE'] = cache_dir
os.environ['HF_HF_HUB_CACHE'] = os.path.join(cache_dir, 'models')

print(f"Cache configurado em: {cache_dir}")

## 5. Executar Avaliação

Agora você pode executar o pipeline de avaliação usando os argumentos do `run.py`. Aqui estão algumas opções:

### Opções de Execução:
- `--quick_eval`: Avaliação rápida com conjunto padrão de perguntas
- `--perguntas "Pergunta 1" "Pergunta 2"`: Lista de perguntas (use aspas para espaços)
- `--ground_truth "Resposta 1" "Resposta 2"`: Respostas ideais (opcional, use "" para vazio)
- `--csv_file arquivo.csv`: Arquivo CSV com perguntas e ground truths
- `--num_queries N`: Número de queries por pergunta (padrão: 3)
- `--modelos modelo1 modelo2`: Lista de modelos a comparar

### Exemplos:
- Avaliação rápida: `!python run.py --quick_eval`
- Com perguntas customizadas: `!python run.py --perguntas "O que é a LGPD?" "Direitos do consumidor" --ground_truth "Lei de proteção de dados" ""`
- Com CSV: `!python run.py --csv_file meu_arquivo.csv`

In [ ]:
# Exemplo: Avaliação rápida
!python run.py --quick_eval --num_queries 2

## Resultados

Após a execução, os resultados serão salvos em:
- `resultados.csv`: Dados tabulares
- `resultados.json`: Dados estruturados
- `comparacao_modelos.json`: Comparação entre modelos
- `report.py`: Gera relatório detalhado

Para visualizar os resultados, execute o relatório:

## 6. Modo de Tratamento de Contexto

O parâmetro `--modo_contexto` controla como o sistema lida com contextos muito grandes que podem exceder o limite de tokens dos modelos:

- **`truncar`** (padrão): Aplica truncamento regressivo do contexto (700k → 100k → 50k → 28k) até que a chamada seja bem-sucedida.
- **`resumir`**: Gera um resumo do contexto usando Gemini 2.5 Flash, preservando detalhes essenciais sem inventar informações.

### Exemplos com Modo de Contexto:
- Avaliação rápida com truncamento: `!python run.py --quick_eval --modo_contexto truncar`
- Avaliação rápida com resumo: `!python run.py --quick_eval --modo_contexto resumir`

In [ ]:
# Exemplo com resumo de contexto
!python run.py --quick_eval --modo_contexto resumir

In [ ]:
# Gerar relatório
!python report.py

In [ ]:
# Mostrar Resultados e Issues
import json
import pandas as pd
from IPython.display import display, Markdown

# Caminhos dos arquivos
results_path = 'results/resultados.json'
comparacao_path = 'results/comparacao_modelos.json'

try:
    # Carregar comparação para tabela e vencedor
    with open(comparacao_path, 'r', encoding='utf-8') as f:
        comparacao = json.load(f)
    
    stats = comparacao.get('estatisticas_por_modelo', {})
    ranking = comparacao.get('ranking_modelos', [])
    
    # Modelo vencedor
    if ranking:
        vencedor = ranking[0]['modelo']
        display(Markdown(f"## 🏆 Modelo Vencedor: **{vencedor}**"))
    
    # Tabela comparativa
    data = []
    for modelo, stat in stats.items():
        data.append({
            'Modelo': modelo,
            'Faithfulness': f"{stat['faithfulness_media']:.3f}",
            'Relevancy': f"{stat['answer_relevancy_media']:.3f}",
            'Context Precision': f"{stat.get('context_precision_media', 0.0):.3f}",
            'ROUGE-1': f"{stat['rouge_1_f1_media']:.3f}",
            'BERTScore': f"{stat['bertscore_f1_media']:.3f}",
            'Tempo Médio (s)': f"{stat['tempo_resposta_medio']:.2f}"
        })
    df = pd.DataFrame(data)
    display(Markdown("## 📊 Comparação de Modelos"))
    display(df)
    
    # Issues identificados
    with open(results_path, 'r', encoding='utf-8') as f:
        resultados = json.load(f)
    
    issues_por_modelo = {}
    for res in resultados:
        modelo = res['modelo']
        issues = res.get('issues', [])
        if issues:
            if modelo not in issues_por_modelo:
                issues_por_modelo[modelo] = []
            issues_por_modelo[modelo].extend(issues)
    
    if issues_por_modelo:
        display(Markdown("## ⚠️ Issues Identificados"))
        for modelo, issues in issues_por_modelo.items():
            display(Markdown(f"**{modelo}:**"))
            for issue in issues:
                display(Markdown(f"- {issue}"))
    else:
        display(Markdown("## ✅ Nenhum Issue Identificado"))

except Exception as e:
    display(Markdown(f"**Erro ao carregar resultados:** {e}"))